In [ ]:
# Exploring conformational space of selected macrocycles - "M13"

In this notebook we present and analyze selected structures, technical notes are [here](www.gitlab.com/user/gosia/icho).

In [88]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
}
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [89]:
import glob
import py3Dmol

from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
from rdkit.Chem import rdMolAlign
from rdkit.Chem.Draw import IPythonConsole
from rdkit import rdBase
print(rdBase.rdkitVersion)
import os,time
print( time.asctime())

2016.09.4
Tue Apr  4 10:47:40 2017


In [90]:
# Functions used in this notebook:

def grep_energies_from_sdf_outputs(files):
    energies = {}
    for inp in files:
        with open(inp,'r') as f:
            lines = f.readlines()
            for i, line in enumerate(lines):
                if "M  END" in line:
                    energies[os.path.splitext(os.path.basename(inp))[0]] = float(lines[i+1])
    return energies

## Crystal structure of "M13" macrocycle

In [91]:
cm13 = open('/home/gosia/work/work_on_gitlab/icho/calcs/m13/m13_crystal.xyz','r').read()
vcm13 = py3Dmol.view(width=400,height=400)
vcm13.removeAllModels()
vcm13.addModel(cm13,'xyz')
vcm13.setStyle({'stick':{'radius':0.15,'color':'spectrum'}})
vcm13.setBackgroundColor('0xeeeeee')
vcm13.zoomTo()
vcm13.show()

In [92]:
# "core" is a part of a molecule, which we wish to be the "most-aligned" among multiple conformers
m13 = Chem.AddHs(Chem.MolFromSmiles('N1C(=O)c2cc(C(=O)NCCC(C)CCNC(=O)c3cc(C(=O)NCCCCC1)cc(c3)C(C)(C)C)cc(c2)C(C)(C)C'))
core_m13 = m13.GetSubstructMatch(Chem.MolFromSmiles('c1ccccc1'))

templ_m13 = Chem.SDMolSupplier('/home/gosia/work/work_on_gitlab/icho/calcs/m13/balloon/m13_crystal.sdf')
m13_crystal = templ_m13[0]

## Conformers generated with the Balloon software:

Conformers were generated in two ways (genetic algorithm):

* Starting with the crystal geometry kept as a template, output: "m13_b_crystal" on the left fig. below

* Starting with the SMILES signature of M13 and allowing to "rebuild the geometry" (option --rebuildGeometry), output: "m13_b_smiles" on the right fig. below

In both cases the Balloon software was asked for 20 conformers (with other parameters set to default values) and it found 15 conformers in the former case and 11 in the latter.

In [93]:
inps_m13_b_sdf = glob.glob('/home/gosia/work/work_on_gitlab/icho/calcs/m13/balloon/results_starting_from_crystalsdf/*.sdf')

In [94]:
inps_m13_b_smi = glob.glob('/home/gosia/work/work_on_gitlab/icho/calcs/m13/balloon/results_starting_from_crystalsmiles/*.sdf')

In [95]:
e_m13_b_sdf = grep_energies_from_sdf_outputs(inps_m13_b_sdf)
e_m13_b_smi = grep_energies_from_sdf_outputs(inps_m13_b_smi)

In [96]:
%%html
<table>
  <tr>
    <td id="m13_b_crystal" ></td>
    <td id="m13_b_smiles"  ></td>
  <tr>
    <td> m13_b_crystal </td>
    <td> m13_b_smiles  </td>  
  </tr>
</table>

,
m13_b_crystal,m13_b_smiles


In [97]:
# write conformers to dictionaries
allmol_m13_b_sdf = {}
allmol_m13_b_smi = {}
suppl_m13_b_sdf  = Chem.SDMolSupplier('/home/gosia/work/work_on_gitlab/icho/calcs/m13/balloon/m13_crystal_sdfout.sdf')
suppl_m13_b_smi  = Chem.SDMolSupplier('/home/gosia/work/work_on_gitlab/icho/calcs/m13/balloon/m13_crystal_smilesout.sdf')

for i, mol in enumerate(suppl_m13_b_sdf):
    name = "m13_b_sdf_" + str(i)
    allmol_m13_b_sdf[name] = mol
for i, mol in enumerate(suppl_m13_b_smi):
    name = "m13_b_smi_" + str(i)
    allmol_m13_b_smi[name] = mol     

In [98]:
# align:
for key, mol in allmol_m13_b_sdf.items():
    AllChem.AlignMolConformers(mol,atomIds=core_m13)
for key, mol in allmol_m13_b_smi.items():
    AllChem.AlignMolConformers(mol,atomIds=core_m13)     

In [99]:
# view:
p13_b_handles=[]

p13_b_sdf = py3Dmol.view(width=400,height=400)
for key, mol in allmol_m13_b_sdf.items():
    mb = Chem.MolToMolBlock(mol)
    p13_b_sdf.addModel(mb,'sdf')
p13_b_sdf.setStyle({'stick':{'radius':'0.1'}})
p13_b_sdf.setBackgroundColor('0xeeeeee')
p13_b_sdf.zoomTo()    
p13_b_handles.append(p13_b_sdf)

p13_b_smi = py3Dmol.view(width=400,height=400)
for key, mol in allmol_m13_b_sdf.items():
    mb = Chem.MolToMolBlock(mol)
    p13_b_smi.addModel(mb,'sdf')
p13_b_smi.setStyle({'stick':{'radius':'0.1'}})
p13_b_smi.setBackgroundColor('0xeeeeee')
p13_b_smi.zoomTo()    
p13_b_handles.append(p13_b_smi)

In [100]:
p13_b_handles[0].insert('m13_b_crystal')

In [101]:
p13_b_handles[1].insert('m13_b_smiles')

### pre-screening

Some of the generated conformers are very much alike. To remove potential duplicates which were not "caught" by the Balloon program, we can compare the energies (preoptimized with MM) and the RMSD calculated against a reference structure (here: the crystal structure of M13).

First let's print the energies and RMS values:

In [102]:
allmol_m13_b = {}
allmol_m13_b.update(allmol_m13_b_sdf)
allmol_m13_b.update(allmol_m13_b_smi)

energy_m13_b = {}
energy_m13_b.update(e_m13_b_sdf)
energy_m13_b.update(e_m13_b_smi)

rms_m13_b = {}
for key, mol in allmol_m13_b.items():
    rms_m13_b[key] = AllChem.GetBestRMS(Chem.RemoveHs(mol),Chem.RemoveHs(m13_crystal))
    print("name = {:>12}, E = {:>.6f}, RMS = {:>.6f}".format(key, energy_m13_b[key], rms_m13_b[key]))

name =  m13_b_sdf_1, E = 119.636517, RMS = 2.478622
name =  m13_b_smi_5, E = 130.489391, RMS = 2.858193
name = m13_b_sdf_12, E = 123.925833, RMS = 2.714899
name = m13_b_sdf_11, E = 123.741071, RMS = 2.372434
name = m13_b_sdf_14, E = 125.798216, RMS = 1.907267
name =  m13_b_smi_6, E = 130.939955, RMS = 2.199209
name =  m13_b_smi_3, E = 127.481759, RMS = 1.832656
name =  m13_b_sdf_8, E = 123.049386, RMS = 2.128462
name =  m13_b_sdf_7, E = 123.026848, RMS = 1.606641
name =  m13_b_smi_4, E = 129.505821, RMS = 2.545563
name =  m13_b_sdf_3, E = 121.413870, RMS = 2.782971
name =  m13_b_smi_1, E = 125.885041, RMS = 2.096618
name =  m13_b_smi_2, E = 126.459585, RMS = 2.128287
name = m13_b_smi_10, E = 134.023181, RMS = 1.852128
name =  m13_b_smi_9, E = 133.192078, RMS = 2.703377
name =  m13_b_smi_7, E = 131.260001, RMS = 2.245091
name =  m13_b_smi_8, E = 131.423063, RMS = 2.562983
name =  m13_b_sdf_5, E = 121.600070, RMS = 2.471335
name =  m13_b_sdf_9, E = 123.621792, RMS = 2.499548
name =  m13_

Then we can introduce some thresholds, for instance:

* if two conformers differ by less than 0.01 in RMS (measured against the reference structure), then select the one with the lower energy

In [103]:
rms_sorted = sorted(rms_m13_b.items(), key=lambda x: x[1])
rms_thresh = 0.01

print("List sorted by RMS:")
for i, t in enumerate(rms_sorted):
    print("i={}, name = {:>12}, E = {:>.6f}, RMS = {:>.6f}".format(i, rms_sorted[i][0], energy_m13_b[rms_sorted[i][0]], rms_sorted[i][1]))

for i, t in enumerate(rms_sorted):
    if i < len(rms_sorted) and i > 0:
        if (rms_sorted[i][1] - rms_sorted[i-1][1]) < rms_thresh:
            print("similar pairs: ", i-1, ": ", rms_sorted[i-1][0], "  and  ", i, ": ", rms_sorted[i][0])
            if energy_m13_b[rms_sorted[i][0]] < energy_m13_b[rms_sorted[i-1][0]]:
                print("removed conformer: ", rms_sorted[i-1][0])
                del energy_m13_b[rms_sorted[i-1][0]]
                del allmol_m13_b[rms_sorted[i-1][0]]
                del rms_m13_b[rms_sorted[i-1][0]]
            else:
                print("removed conformer: ", rms_sorted[i][0])
                del energy_m13_b[rms_sorted[i][0]]
                del allmol_m13_b[rms_sorted[i][0]]                
                del rms_m13_b[rms_sorted[i][0]]  

List sorted by RMS:
i=0, name =  m13_b_sdf_0, E = 112.637348, RMS = 0.877066
i=1, name =  m13_b_smi_0, E = 120.570942, RMS = 1.543718
i=2, name =  m13_b_sdf_7, E = 123.026848, RMS = 1.606641
i=3, name =  m13_b_smi_3, E = 127.481759, RMS = 1.832656
i=4, name = m13_b_smi_10, E = 134.023181, RMS = 1.852128
i=5, name = m13_b_sdf_14, E = 125.798216, RMS = 1.907267
i=6, name =  m13_b_smi_1, E = 125.885041, RMS = 2.096618
i=7, name = m13_b_sdf_13, E = 124.181713, RMS = 2.100488
i=8, name =  m13_b_smi_2, E = 126.459585, RMS = 2.128287
i=9, name =  m13_b_sdf_8, E = 123.049386, RMS = 2.128462
i=10, name = m13_b_sdf_10, E = 123.713041, RMS = 2.137413
i=11, name =  m13_b_smi_6, E = 130.939955, RMS = 2.199209
i=12, name =  m13_b_sdf_4, E = 121.467767, RMS = 2.224798
i=13, name =  m13_b_smi_7, E = 131.260001, RMS = 2.245091
i=14, name = m13_b_sdf_11, E = 123.741071, RMS = 2.372434
i=15, name =  m13_b_sdf_5, E = 121.600070, RMS = 2.471335
i=16, name =  m13_b_sdf_1, E = 119.636517, RMS = 2.478622
i=17

Below we will align the selected conformers:

In [104]:
for key, mol in allmol_m13_b.items():
    core_mol = mol.GetSubstructMatch(Chem.MolFromSmiles('c1ccccc1'))
    AllChem.AlignMol(mol,m13_crystal,atomMap=list(zip(core_mol,core_m13)))
    
p_b = py3Dmol.view(width=400,height=400)
for key, mol in allmol_m13_b.items():
    mb = Chem.MolToMolBlock(mol)
    p_b.addModel(mb,'sdf')
p_b.setStyle({'stick':{'radius':'0.15'}})
p_b.setBackgroundColor('0xeeeeee')
p_b.zoomTo()
p_b.show()